In [6]:
!pip install gradio
!pip install openai
!pip install langchain openai
!pip install langchain
!pip install langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.4 MB/s eta 0:00:00


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6d24d22b45e6453489.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [13]:
!pip install tensorflow

In [118]:
from PIL import Image
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import img_to_array


def predictFoodItem(imagePath):

  class_labels = getClassLabels(directory_T)
  food_model = loadModel()
  image_array = getImage(imagePath)

  # Make a prediction
  predictions = food_model(image_array)

  # Now find the predicted class
  predicted_class = np.argmax(predictions, axis=0)
  class_labels = tr_d['label'].unique().tolist()
  foodItem  = class_labels[predicted_class]

  return foodItem



In [121]:
# Load Data and Make DataFrame
import os
import pandas as pd
def getClassLabels(directory):
    filepath =[]
    label = []

    folds = os.listdir(directory)

    for fold in folds:
        f_path = os.path.join(directory , fold)
        imgs = os.listdir(f_path)
        for img in imgs:
            img_path = os.path.join(f_path , img)
            filepath.append(img_path)
            label.append(fold)

    #Concat data paths with labels
    file_path_series = pd.Series(filepath , name= 'filepath')
    Label_path_series = pd.Series(label , name = 'label')
    df_train = pd.concat([file_path_series ,Label_path_series ] , axis = 1)

    class_labels = df_train['label'].unique().tolist()

    return class_labels

# # Directory containing the "Train" folder
directory_T = "/content/drive/MyDrive/fast-food-classification-dataset/Fast Food Classification V2/Train"

class_labels = getClassLabels(directory_T)

class_labels

['Baked Potato',
 'Burger',
 'Crispy Chicken',
 'Donut',
 'Fries',
 'Hot Dog',
 'Pizza',
 'Sandwich',
 'Taco',
 'Taquito']

In [131]:
#!pip install tensorflow
import tensorflow as tf
from tensorflow import keras


def loadModel():
    # Load the SavedModel as a TFSMLayer
    food_model = tf.keras.layers.TFSMLayer('/content/drive/MyDrive/fast-food-classification-dataset/MyModel/food_imageprocessing_model', call_endpoint='serving_default')
    #food_model = load_model('/content/drive/MyDrive/fast-food-classification-dataset/MyModel/food_imageprocessing_model')
    # Note: Replace 'serving_default' with the correct call_endpoint if it's different
    return food_model


In [132]:
def getImage(imagePath):

  image = Image.open(imagePath)
  image = image.resize((256, 256))
  image_array = img_to_array(image)
  image_array = np.expand_dims(image_array, axis=0)
  image_array = image_array /  255.0

  return image_array

In [133]:
# Load and preprocess the image

#image_path ='/content/drive/MyDrive/fast-food-classification-dataset/Fast Food Classification V2/Train/Fries/Fries-Train (1).jpeg'
image_path ='/content/drive/MyDrive/fast-food-classification-dataset/Fast Food Classification V2/Train/Donut/Donut (1).jpeg'
#image_path = '/content/drive/MyDrive/fast-food-classification-dataset/Fast Food Classification V2/Train/Burger/Burger-Train (100).jpeg'


predictFoodItem(image_path)

'Baked Potato'

In [134]:
import openai
from langchain.chat_models import ChatOpenAI

API_KEY = "API_KEY"
OPENAI_MODEL = "gpt-3.5-turbo" # Add the model name you want to use

def chatgpt( image, input_text):

    foodItem = predictFoodItem(image)

    #input_text='vanilla cake nutrition facts'
    llm = ChatOpenAI(openai_api_key=API_KEY, model_name=OPENAI_MODEL, temperature=0.3) # Now ChatOpenAI is defined
    # Pass the query to the invoke method which will make a prediction based on the query, and print the result.
    result = llm.invoke(foodItem+" "+ input_text)

    # Return the generated text
    return foodItem + ' '+ input_text +' ## '+ result.content + ' ## ' + image

#test = chatgpt('Hello')

#print(test)

In [135]:
#Create the Gradio app
demo = gr.Interface(
    fn=chatgpt,
    inputs=[
        gr.File(label="Upload the picture of food you want to know about:"),
        gr.Textbox(label="Enter your message")
    ],
    outputs=gr.Textbox(label="Here are some details about it"),
    title="AI NutriSnap ",
    description="Snap a photo, get the nutrition facts.",
)

# Launch the app
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1e3a2c94acca81eda2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
